In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

C:\Users\Master\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Master\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Master\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import pandas as pd
intents = pd.read_json('mentalhealth.json')
intents

,intents
0,"{'tag': 'definition', 'patterns': ['What does ..."
1,"{'tag': 'affects_whom', 'patterns': ['Who does..."
2,"{'tag': 'what_causes', 'patterns': ['What caus..."
3,"{'tag': 'recover', 'patterns': ['Can people wi..."
4,"{'tag': 'steps', 'patterns': ['I know someone ..."
5,"{'tag': 'find_help', 'patterns': ['How to find..."
6,"{'tag': 'treatement_options', 'patterns': ['Wh..."
7,"{'tag': 'treatment_tips', 'patterns': ['How to..."
8,"{'tag': 'professional_types', 'patterns': ['Wh..."
9,"{'tag': 'right_professional', 'patterns': ['Ho..."


In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

22 documents
10 classes ['affects_whom', 'definition', 'find_help', 'professional_types', 'recover', 'right_professional', 'steps', 'treatement_options', 'treatment_tips', 'what_causes']
64 unique lemmatized words ['a', 'affect', 'affected', 'appears', 'are', 'available', 'be', 'become', 'begin', 'between', 'by', 'can', 'cause', 'describe', 'difference', 'different', 'doe', 'find', 'followed', 'for', 'from', 'get', 'have', 'health', 'how', 'i', 'if', 'ill', 'illness', 'in', 'incase', 'involved', 'is', 'it', 'keep', 'know', 'lead', 'mean', 'mental', 'mentall', 'mentally', 'mind', 'myself', 'of', 'one', 'option', 'people', 'possible', 'present', 'professional', 'recover', 'right', 'should', 'someone', 'step', 'such', 'symptom', 'the', 'to', 'treatment', 'type', 'what', 'who', 'with']


In [7]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [8]:
random.shuffle(training)


In [9]:
training = np.array(training,dtype = 'object')

In [11]:
from tensorflow.keras.optimizers import SGD
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

#fitting and saving the model 
histogram = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
#model.save('mental_chatbot_model.keras', histogram)

Training data created
Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.1359 - loss: 2.4222
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0346 - loss: 2.4996     
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1359 - loss: 2.2446 
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2520 - loss: 2.2511 
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2619 - loss: 2.1377 
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 2.2287 
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3283 - loss: 2.0348 
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1467 - loss: 2.2550 
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3795 - loss: 2.1101 
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4295 - loss: 1.9536 
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4808 - loss: 1.9400 
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.3175 
Epoch 70/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9487 - loss: 0.1560 
Epoch 71/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9765 - loss: 0.2349 
Epoch 72/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8255 - loss: 0.4480 
Epoch 73/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9848 - loss: 0.3125 
Epoch 74/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8975 - loss: 0.4426 
Epoch 75/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9253 - loss: 0.3196 
Epoch 76/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9765 - loss: 0.3234 
Epoch 77/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.1484 
Epoch 78/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.2519 
Epoch 79/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9654 - loss: 0.3213 
Epoch 80/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9765 - loss: 0

In [15]:
model.save('mental_chatbot_model.keras', histogram)

In [16]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, histogram):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [20]:

def chat_with_chatbot():
    print("Start talking with the chatbot (type 'quit' to stop)!")
    while True:
        sentence = input("You: ")
        if sentence.lower() == 'quit':
            break
        ints = predict_class(sentence, model)
        res = getResponse(ints, intents)
        print("Chatbot: " + res)

if __name__ == "__main__":
    chat_with_chatbot()

Start talking with the chatbot (type 'quit' to stop)!
You: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


IndexError: list index out of range